In [1]:
from dotenv import load_dotenv
load_dotenv()

True

1) first complete remaining part of this notebook
2) Take multiple PDF with text, image, table
3) first fetch data from pdf
4) atleast there should be 200 pages
5) if chunking(use semantic chunking) required do chunking and then embedding ## I have not discussed
6) store it inside the vector DB (use any of them mongodb, astradb, opensearch, milvus/quadrant)## I have not discussed
7) create a index with all three index mechanism(flat index, HNSW, IVF) ## I have not discussed
8) create a retriever pipeline
9) check the retriever time (which one is fastest)
10) print the accuracy score of every similarity search
11) perform reranking using BM25 or MMR ## I have not discussed
12) write a prompt template and generate a output through llm
13) render that output over the DOCx ## I have not discussed
14) as an additional tip : follow rag playlist from youtube channel

In [2]:
import os
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [3]:
from langchain_community.document_loaders import PyPDFLoader
FILE_PATH = "111382_S_Chand_Annual_Report_2019.pdf"
loader = PyPDFLoader(FILE_PATH)

In [4]:
docs = loader.load()

In [6]:
docs

[Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2019-08-23T19:12:52+05:30', 'moddate': '2019-08-23T19:13:54+05:30', 'trapped': '/False', 'source': '111382_S_Chand_Annual_Report_2019.pdf', 'total_pages': 232, 'page': 0, 'page_label': 'i'}, page_content='Renewed F o cus.\nAnnual Report 2018-2019'),
 Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2019-08-23T19:12:52+05:30', 'moddate': '2019-08-23T19:13:54+05:30', 'trapped': '/False', 'source': '111382_S_Chand_Annual_Report_2019.pdf', 'total_pages': 232, 'page': 1, 'page_label': 'ii'}, page_content='This report may contain "forward looking statements" by S.Chand and Company Limited. These may include statements \nrelating to future results of operations, financial condition, business prospects, plans and objectives, and are based on the \ncurrent beliefs, assumptions, expectations, estimates, and p

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [9]:
splitted_docs = splitter.split_documents(docs)

In [11]:
len(splitted_docs)

1797

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')

/Users/s0s14tr/Desktop/learning/krish_agentic/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from langchain_milvus import Milvus
URI = "./milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "IVF_FLAT", "metric_type": "L2"},
)

In [16]:
vector_store.add_documents(splitted_docs)

[460023892571586560,
 460023892571586561,
 460023892571586562,
 460023892571586563,
 460023892571586564,
 460023892571586565,
 460023892571586566,
 460023892571586567,
 460023892571586568,
 460023892571586569,
 460023892571586570,
 460023892571586571,
 460023892571586572,
 460023892571586573,
 460023892571586574,
 460023892571586575,
 460023892571586576,
 460023892571586577,
 460023892571586578,
 460023892571586579,
 460023892571586580,
 460023892571586581,
 460023892571586582,
 460023892571586583,
 460023892571586584,
 460023892571586585,
 460023892571586586,
 460023892571586587,
 460023892571586588,
 460023892571586589,
 460023892571586590,
 460023892571586591,
 460023892571586592,
 460023892571586593,
 460023892571586594,
 460023892571586595,
 460023892571586596,
 460023892571586597,
 460023892571586598,
 460023892571586599,
 460023892571586600,
 460023892571586601,
 460023892571586602,
 460023892571586603,
 460023892571586604,
 460023892571586605,
 460023892571586606,
 460023892571

In [ ]:
results = vector_store.similarity_search(
    "what is revenue of schand?",
    k=2
)

In [18]:
results

[Document(metadata={'creationdate': '2019-08-23T19:12:52+05:30', 'creator': 'Adobe InDesign CS6 (Windows)', 'moddate': '2019-08-23T19:13:54+05:30', 'page': 63, 'page_label': '62', 'pk': 460023892571586969, 'producer': 'Adobe PDF Library 10.0.1', 'source': '111382_S_Chand_Annual_Report_2019.pdf', 'total_pages': 232, 'trapped': '/False'}, page_content="fees and reimburse travel expenses for attending the Board and Committee\xa0meetings.\n2. Shareholder Rights The quarterly and half-yearly financial results along with the press release are published in \nthe newspapers and also uploaded on the website of the Company at www.schandgroup.com.\n3. Audit Qualifications The auditor’s report of the Company for the financial year 2018-19 contains qualification as \ndetailed in the Board's Report"),
 Document(metadata={'creationdate': '2019-08-23T19:12:52+05:30', 'creator': 'Adobe InDesign CS6 (Windows)', 'moddate': '2019-08-23T19:13:54+05:30', 'page': 24, 'page_label': '23', 'pk': 460023892571586

In [19]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [20]:
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(model = "o1-mini")

In [21]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [22]:
def format_docs(docs):
    return "/n /n".join(doc.page_content for doc in docs)

In [23]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| model
| StrOutputParser()
)

In [26]:
rag_chain.invoke("what are some inorganic digital investments made by schand?")


/Users/s0s14tr/Desktop/learning/krish_agentic/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


'S Chand made inorganic digital investments by granting optionally convertible loans of ₹50 million each to DS Digital Private Limited and Safari Digital Education Initiatives Private Limited, totaling ₹100 million. Additionally, the company invested ₹39.95 million in its wholly-owned subsidiary, S Chand Edutech Private Limited. These investments support S Chand’s expansion into the digital education sector.'